# 3.决策树
## 3.1.原理

决策树是一种监督式分类方法。

在一个类型和特征值已知的结构化数据集上，根据数据类型的概率，建立一个树形结构，用于数据分类。

训练一个决策树，需要在每次划分时，以最大信息增益的特征进行划分。

使用一个决策树预测，需要从树的根节点，一步步进行判断，最后在叶节点得到预测的分类。

## 调包

In [3]:
import numpy as np
import operator

## 3.2.信息增益的计算
### 3.2.1.建立结构化数据

In [8]:
dataSet = [['sunny', 'high', 'false', 'hot', 'yes'],
          ['sunny', 'high', 'false', 'hot', 'yes'],
          ['overcast', 'high', 'false', 'mild', 'yes'],
          ['overcast', 'normal', 'false', 'mild', 'yes'],
          ['overcast', 'normal', 'false', 'mild', 'yes'],
          ['overcast', 'normal', 'false', 'mild', 'yes'],
          ['rainy', 'normal', 'true', 'cool', 'yes'],
          ['rainy', 'normal', 'true', 'cool', 'yes'],
          ['rainy', 'normal', 'true', 'cool', 'yes'],
          ['sunny', 'high', 'false', 'hot', 'no'],
          ['sunny', 'high', 'false', 'hot', 'no'],
          ['sunny', 'high', 'true', 'mild', 'no'],
          ['rainy', 'high', 'true', 'mild', 'no'],
          ['rainy', 'normal', 'true', 'cool', 'no']]
features = ['outlook', 'humidity', 'windy', 'temperature']

### 3.2.2.计算信息增益的函数
信息量为概率的负对数，概率越小，信息量越大
$$l(x_{i})=-log_{2}p(x_{i})$$
香农熵为信息量的期望，统计平均
$$H=E[l(x_{i})]=-\sum_{i=1}^{n}p(x_{i})\cdot log_{2}p(x_{i})$$
信息增益为划分前的信息熵减去划分后的各个子类信息熵之和
$$G=H_{orig}-\sum_{i=1}^{K}H_{divd_{i}}$$

In [5]:
from math import log

def calcShannonEnt(dataSet):
    """
    计算香农熵
    参数：
        dataSet -- 数据集，包含不同类型
    输出：
        shannonEnt -- 香农熵
    """
    #求数据集大小
    numEntries = len(dataSet)
    #新建一个用于统计各个类型数目的字典
    labelCounts = {}
    #遍历数据集中的每个数据
    for featVec in dataSet:
        #类型为数据中的最后一项
        currentLabel = featVec[-1]
        #如果之前没有记录该类型
        if currentLabel not in labelCounts.keys():
            #添加新的一项记录
            labelCounts[currentLabel] = 0
        #计数
        labelCounts[currentLabel] += 1
    #初始化香农熵
    shannonEnt = 0.0
    #遍历所有类型
    for key in labelCounts:
        #计算概率，古典概型
        prob = np.float(labelCounts[key])/numEntries
        #根据公式计算香农熵
        shannonEnt -= prob * log(prob, 2)
    return shannonEnt

In [10]:
def chooseBestFeatureToSplit(dataSet, features):
    """
    选择最好的划分参照特征，也就是使得信息增益最大的划分方式
    参数：
        dataSet -- 数据集
        features -- 特征名
    返回：
        bestFeature -- 最好的参照类
        infoGains -- 信息增益，字典
    """
    #建立一个字典，存储信息增益
    infoGains = {}
    #求特征数，-1是去掉标签
    numFeatures = len(dataSet[0]) - 1
    #求数据集的香农熵
    baseEntropy = calcShannonEnt(dataSet)
    #初始化最佳信息增益，和最好的参照特征
    bestInfoGain = 0.0
    bestFeature = -1
    #遍历每个特征
    for i in range(numFeatures):
        #取每个数据的第i个特征值
        featList = [example[i] for example in dataSet]
        #把这些特征值作为一个集合
        uniqueVals = set(featList)
        #初始化划分后的熵
        newEntropy = 0.0
        #遍历集合中的特征值
        for value in uniqueVals:
            #取第i个特征值为value子集
            subDataSet = splitDataSet(dataSet, i, value)
            #计算概率
            prob = len(subDataSet) / np.float(len(dataSet))
            #计算划分后的熵，为各个子集熵的期望
            newEntropy += prob * calcShannonEnt(subDataSet)
        #计算信息增益
        infoGain = baseEntropy - newEntropy
        #加入列表
        infoGains[features[i]] = infoGain
        #如果当前以i特征划分得到的信息增益大于之前的最大值
        if (infoGain > bestInfoGain):
            #加冕为王
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature, infoGains

In [11]:
bestFeature, infoGains = chooseBestFeatureToSplit(dataSet, features)
print("所有特征的信息增益是：\n{}\n".format(infoGains))
print("最好的分类特征是——{}".format(features[bestFeature]))

所有特征的信息增益是：
{'outlook': 0.2467498197744391, 'humidity': 0.15183550136234136, 'windy': 0.04812703040826927, 'temperature': 0.029222565658954647}

最好的分类特征是——outlook
